In [0]:
spark

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# create data
data = [
    ("Aman", "Gupta", 23, "Delhi"),
    ("Sachin", "Tendulkar", 32, "Mumbai"),
    ("Rahul", "Dravid"   , 35, "Bangalore"),    
    ("Virat", "Kohli", 30, "Mumbai"),
    ("Saurav", "Ganguly", 31, "Mumbai"),
    ("Rajesh", None, 33, "Bangalore"),
]

In [0]:
# defining schema
schema = StructType([
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("city", StringType(), True)
])

In [0]:
df = spark.createDataFrame(data = data, schema = schema)

In [0]:
df.show()

In [0]:
%sql
select * from df

In [0]:
df.withColumn("last_name", when(col("last_name") == "", None).otherwise(col("last_name")))

In [0]:
df.show()

In [0]:
df.filter(col("last_name").isNull()).show()

In [0]:
df = df.withColumnRenamed("last_name", "last")

In [0]:
df.show()

In [0]:
df2 = df.filter(col("first_name")==0)

In [0]:
df2.show()

In [0]:
df = df.withColumn("age", col("age").cast(IntegerType()))

In [0]:
df.select("*").show()

In [0]:
# df.printSchema()
df.schema["age"].dataType

In [0]:
# df2 = spark.read.csv("C:\\Users\\Ketan.Singh\\Downloads\\sales.csv")

# Delta tables

In [0]:
from delta.tables import DeltaTable

In [0]:
DeltaTable.createIfNotExists(spark)\
    .tableName("dt")\
    .addColumn("first_name", "string")\
    .addColumn("last_name", "string")\
    .addColumn("age", "int")\
    .addColumn("city", "string")\
    .execute()

In [0]:
spark.sql("select * from dt")

In [0]:
df.select("*").show()

In [0]:
df2 = spark.createDataFrame(
    [("Ankur", "Singh", 45, "Delhi", "India"),
     ("Sahil", "Dhillon", 15, "Noida", "India")],
    ["first_name", "last_namee", "age", "city", "country"]
)

In [0]:
df2.show()

In [0]:
df.write.format("delta").saveAsTable("df_delta")

In [0]:
spark.sql("select * from df_delta").show()

In [0]:
df_delta_read = spark.read.format("delta").table("df_delta")

df_delta_read.alias("target").merge(
    df.alias("source"), "source.first_name = target.first_name"
).whenMatchedUpdateAll()\
 .whenNotMatchedInsertAll()\
 .execute()

In [0]:
spark.sql(
    '''
        insert into dt values
        ("Aman", "Sharma", 23, "Patiala"),
        ("Raj", "Singh", 22, "Ludhiana")
    '''
)

In [0]:
df2 = spark.createDataFrame(
    [("Ankur", "Singh", 45, "Delhi", "India"),
     ("Sahil", "Dhillon", 15, "Noida", "India")],
    ["first_name", "last_namee", "age", "city", "country"]
)

In [ ]:
df2.show()

In [0]:
df2.write.format("delta").mode("append").saveAsTable("dt")

In [0]:
df2.write.option("mergeSchema", "True").format("delta").mode("append").saveAsTable("dt")

In [0]:
spark.sql("select * from dt")